# Natural Langage processing

## Bag of Words process

In [1]:
vocab = {} # maps word to integer representing it
word_encoding =1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(' ') # create a list of all of the words in the text, assuming there is no grammer in our text example
  bag = {} # store all of the encodings and their frequency

  for word in words:
    if word in vocab: 
      encoding = vocab[word] # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1

    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1

  return bag

text = 'this is a test to see if this test will work is is test c c'
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 1, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 2}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9, 'c': 10}


## Integer Encoding
### Help representing each word or character in a sentence as unique integer and maintaining the order of these words

In [2]:
vocab ={}
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(' ')
  encoding = []

  for word in words:
    if word in vocab:
      code = vocab[word]
      encoding.append(code)
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1

  return encoding


text = 'this is a test to see if this test will work is is test c c'
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 10, 10]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9, 'c': 10}


# Movie review Dataset
### Load IMDB movie review datasets from keras. It contains 25000 reviews from IMDB where each one is already preprocessed and has label either positive or negative. Each review is encoded by integers that represents how common a word is in the entire dataset. For example, a word encoded by the integer 3 means that it is the 3rd most common word in the dataset.

In [3]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

17473536/17464789 [==============================] - 0s 0us/step


In [4]:
# check one of the reviews
train_data[1]

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 10156,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 11596,
 349,
 2637,
 148,
 605,
 15358,
 8003,
 15,
 123,
 125,
 68,
 23141,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 36893,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 25249,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 46151,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 

Processing to standardised each reveiws length ie if reveiwe length more than 250 words, it will trim and if review length is less than 250 words, it will add dummy words ie padding.

keras has predefined function to do so

In [5]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

### creating model 
Now it's time to create the model. We'll use a word embedding layer as the first layer in our model and add a LSTM layer afterwards that feeds into a dense node to get our predicted sentiment. 

32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like!

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


##Training

In [8]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 17s 12ms/step - loss: 0.4126 - acc: 0.8120 - val_loss: 0.3049 - val_acc: 0.8800
Epoch 2/10
625/625 [==============================] - 7s 11ms/step - loss: 0.2360 - acc: 0.9086 - val_loss: 0.3139 - val_acc: 0.8742
Epoch 3/10
625/625 [==============================] - 7s 11ms/step - loss: 0.1857 - acc: 0.9323 - val_loss: 0.3349 - val_acc: 0.8708
Epoch 4/10
625/625 [==============================] - 7s 11ms/step - loss: 0.1563 - acc: 0.9443 - val_loss: 0.3058 - val_acc: 0.8902
Epoch 5/10
625/625 [==============================] - 7s 11ms/step - loss: 0.1297 - acc: 0.9554 - val_loss: 0.2887 - val_acc: 0.8816
Epoch 6/10
625/625 [==============================] - 7s 11ms/step - loss: 0.1147 - acc: 0.9609 - val_loss: 0.2994 - val_acc: 0.8864
Epoch 7/10
625/625 [==============================] - 7s 11ms/step - loss: 0.1000 - acc: 0.9668 - val_loss: 0.3568 - val_acc: 0.8840
Epoch 8/10
625/625 [==============================] - 7s 11ms/step -

In [9]:
# evaluation of model using test data
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 4s 6ms/step - loss: 0.4561 - acc: 0.8570
[0.45605039596557617, 0.8569599986076355]


###Making Predictions
Now let’s use our network to make predictions on our own reviews. 

Since our reviews are encoded well need to convert any review that we write into that form so the network can understand it. To do that well load the encodings from the dataset and use them to encode our own data.


In [10]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = 'that movie was just amazing, really great'
encoded = encode_text(text)
print(encoded)

1654784/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [11]:
# code to decode word index to given integer output as shown above

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
  PAD =0
  text =''
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num] + ' '

  return text[:-1]

print(decode_integers(encoded))

that movie was just amazing really great


In [12]:
# time to make some predictions

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1, 250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.83452684]
[0.47138208]
